In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scipy.stats import zscore

In [ ]:
# Create Random data
np.random.seed(42)
normal_data = np.random.normal(0, 1, (1000, 2))
anomalous_data = np.random.uniform(-4, 4, (50, 2))

data = np.concatenate((normal_data, anomalous_data), axis=0)
labels = np.concatenate((np.zeros(1000), np.ones(50)), axis=0)  # 0: normal, 1: anomaly

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Normalize data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
def z_score_method(data):
    z_scores = np.abs(zscore(data))
    return (z_scores > 3).any(axis=1).astype(int)  # Anomaly if any z-score > 3


In [ ]:
iso_forest = IsolationForest(contamination=0.05, random_state=42)
iso_forest.fit(X_train)
y_pred_iso_forest = iso_forest.predict(X_test)
y_pred_iso_forest = np.where(y_pred_iso_forest == -1, 1, 0)  # Convert to 1 for anomaly


In [ ]:
# Define Autoencoder model
input_dim = X_train.shape[1]
autoencoder = Sequential([
    Dense(16, activation='relu', input_shape=(input_dim,)),
    Dense(8, activation='relu'),
    Dense(16, activation='relu'),
    Dense(input_dim, activation='sigmoid')
])

autoencoder.compile(optimizer='adam', loss='mse')

# Train Autoencoder
autoencoder.fit(X_train, X_train, epochs=50, batch_size=32, validation_split=0.1, verbose=0)

# Calculate reconstruction error
reconstructions = autoencoder.predict(X_test)
mse = np.mean(np.power(X_test - reconstructions, 2), axis=1)
y_pred_autoencoder = (mse > np.percentile(mse, 95)).astype(int)  # Top 5% as anomalies


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step


In [ ]:
# Z-Score
y_pred_z_score = z_score_method(X_test)

# Combine predictions using majority voting
y_pred_combined = np.array([
    (z + iso + ae) > 1  # Majority voting (2 out of 3)
    for z, iso, ae in zip(y_pred_z_score, y_pred_iso_forest, y_pred_autoencoder)
]).astype(int)


In [ ]:
# Evaluation metrics
print("Accuracy:", accuracy_score(y_test, y_pred_combined))
print("Precision:", precision_score(y_test, y_pred_combined))
print("Recall:", recall_score(y_test, y_pred_combined))
print("F1 Score:", f1_score(y_test, y_pred_combined))


Accuracy: 0.9714285714285714
Precision: 0.8181818181818182
Recall: 0.6923076923076923
F1 Score: 0.7500000000000001
